# Student Loan Risk with Deep Learning

In [291]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
# from numpy import np

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [292]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.describe()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,0.534709
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.498950
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,0.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,0.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,1.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,1.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,1.000000


In [293]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [294]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [295]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

X = loans_df.drop(columns='credit_ranking')


# Display a sample of y
y.values[:5]

array([0, 0, 0, 1, 0], dtype=int64)

In [296]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns='credit_ranking')

# Review the features DataFrame
X.describe()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000


In [297]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Step 3: Split the features and target sets into training and testing datasets.


In [298]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [299]:
# Create a StandardScaler instance
ss = StandardScaler()

# Fit the scaler to the features training dataset
ss.fit(X_train)
# ss.fit(X_test)
# Fit the scaler to the features test dataset
X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)



In [300]:
# X_train_scaled.value_counts()

In [301]:
nn_model = tf.keras.models.Sequential()

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [302]:
nn_model = tf.keras.models.Sequential()

# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   payment_history         1599 non-null   float64
 1   location_parameter      1599 non-null   float64
 2   stem_degree_score       1599 non-null   float64
 3   gpa_ranking             1599 non-null   float64
 4   alumni_success          1599 non-null   float64
 5   study_major_code        1599 non-null   float64
 6   time_to_completion      1599 non-null   float64
 7   finance_workshop_score  1599 non-null   float64
 8   cohort_ranking          1599 non-null   float64
 9   total_loan_score        1599 non-null   float64
 10  financial_aid_score     1599 non-null   float64
dtypes: float64(11)
memory usage: 137.5 KB


In [303]:
# Define the number of hidden nodes for the first hidden layer
layer1 = 21

# Define the number of hidden nodes for the second hidden layer
layer2 = 10



# Define the number of neurons in the output layer
layer3 = 8

In [304]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=layer1, activation="relu", input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=layer2, activation="relu"))

# Add the third hidden layer
nn_model.add(tf.keras.layers.Dense(units=layer3, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [305]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 21)                252       
                                                                 
 dense_49 (Dense)            (None, 10)                220       
                                                                 
 dense_50 (Dense)            (None, 8)                 88        
                                                                 
 dense_51 (Dense)            (None, 1)                 9         
                                                                 
Total params: 569
Trainable params: 569
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [306]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])



In [307]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
40/40 [==============================] - 0s 4ms/step - loss: 0.7007 - accuracy: 0.5027
Epoch 2/50
40/40 [==============================] - 0s 2ms/step - loss: 0.6384 - accuracy: 0.6231
Epoch 3/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5970 - accuracy: 0.6959
Epoch 4/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5654 - accuracy: 0.7279
Epoch 5/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5438 - accuracy: 0.7263
Epoch 6/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5310 - accuracy: 0.7318
Epoch 7/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5214 - accuracy: 0.7490
Epoch 8/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5148 - accuracy: 0.7467
Epoch 9/50
40/40 [==============================] - 0s 1ms/step - loss: 0.5112 - accuracy: 0.7482
Epoch 10/50
40/40 [==============================] - 0s 1ms/step - loss: 0.5034 - accuracy: 0.7514
Epoch 11/50
40/40 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [308]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)


# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - loss: 0.5132 - accuracy: 0.7594 - 54ms/epoch - 5ms/step
Loss: 0.5131984949111938, Accuracy: 0.7593749761581421


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [309]:
# Set the model's file path
model_file_path = "student_loans.keras"

# Export your model to a keras file
nn_model.save(model_file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [310]:
from tensorflow.keras.models import load_model
# Set the model's file path
model_file_path = "student_loans.keras"

# Load the model to a new object
loaded_nn_model = load_model(model_file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [311]:
# Make predictions with the test data
# ss.fit(X_test)
# newtest = ss.transform(X_test)

predictions = loaded_nn_model.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
predictions[0:5]

10/10 - 0s - 39ms/epoch - 4ms/step


array([[0.25214222],
       [0.3202116 ],
       [0.74137354],
       [0.6921812 ],
       [0.99748945]], dtype=float32)

In [312]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=["predictions"], data=predictions)
predictions_df["predictions"] = round(predictions_df["predictions"],0)
predictions_df


,predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
315,0.0
316,0.0
317,0.0
318,1.0


### Step 4: Display a classification report with the y test data and predictions

In [313]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.72      0.77      0.74       154
           1       0.77      0.73      0.75       166

    accuracy                           0.75       320
   macro avg       0.75      0.75      0.75       320
weighted avg       0.75      0.75      0.75       320



---
## Discuss creating a recommendation system for student loans

In [314]:
#1:
 # Age of the borrower would be good to collect. Older people usually are employed and have credit history compared to younger people who just enter the college sphere. 
 # If the individual is young with no credit history then it would default to what major they are going for. If the major has a high income then the loan would have a higher chance of being approved.
 # This could potentially lead to discrimination against younger people though so it would have to be monitored. 
 # Another good one would be the amount of the loan being requested. Do higher loans get rejected more than smaller loans or is it dependant on the type of major being studied?
 # The type of school that the individual will be attending. Is it a university or a junior college? Maybe get a college score for success depending on how many students graduate and find gainful employment

#2:
 # I think collaborative based filtering would be the best for this. Since a lot of borrowers might be young it should look at other borrowers with the same profiles to see if it would be a good idea to recommend a loan or not.
 # If a bunch of people with the same major going to the same school default on a loan then it would deny a loan to that person. 

 # Another option would be to look at the content of the user for Debt to income but some people may not have that established yet.

#3:
 # I think discrimination would be the biggest challenge. In the quest to approve or deny loans it could inadvertently deny to more borrowers if a high amount of people denied happen to be a certain race or gender even though that isnt listed.\
 # If age was implemented it could deny based off age and only borrow to older users.
 # Anyone could default on a loan even someone with high probability of paying it back so it wouldnt be perfect.

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

